In [ ]:
import numpy as np
import pandas as pd 

# Carregar dados

In [ ]:
# Vamos carregar apenas 200000 linhas para experimentação

df = pd.read_csv("../input/jane-street-market-prediction/train.csv", nrows=200000)

In [ ]:
df.head()

# Engenharia de atributos

A ideia simples aqui é apenas analisar cada dia e decidir por uma ação. Em cada dia, fazemos uma ação se aquele dia tiver "lucro" -- weight*resp for positivo.

In [ ]:
# Preencher os NaN com 0, mas deve haver algo melhor para fazer aqui

df = df.fillna(0)

In [ ]:
# Criar o atributo-alvo

df["profit"] = df["weight"]*df["resp"] > 0
df.groupby('profit').size()

# Treinar um modelo

In [ ]:
training_features = [atributo for atributo in df.columns if "feature_" in atributo]

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

X = df[training_features]
y = df["profit"]
data_dmatrix = xgb.DMatrix(data=X,label=y)

# Criar conjunto de teste. Temos que criar de forma ordenada temporalmente (treino = passado, teste = futuro)

train_size = 0.8
X_train = X[0:int(train_size*len(X))]
X_test = X[int(train_size*len(X)):]
y_train = y[0:int(train_size*len(X))]
y_test = y[int(train_size*len(X)):]

In [ ]:
xgb_clf = xgb.XGBClassifier(objective ='reg:logistic')
xgb_clf.fit(X_train,y_train)

# Avaliar o modelo treinado

In [ ]:
from sklearn.metrics import classification_report

preds = xgb_clf.predict(X_test)
print(classification_report(y_test, preds))

# Treinar o modelo com todos dados

In [ ]:
xgb_clf = xgb.XGBClassifier(objective ='reg:logistic')
xgb_clf.fit(X,y)

# Predições e submissão

In [ ]:
# https://www.kaggle.com/c/jane-street-market-prediction/overview/evaluation

import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.fillna(0)
    X = test_df[training_features]
    pred = xgb_clf.predict(X)
    sample_prediction_df.action = pred.astype(int)
    env.predict(sample_prediction_df)